In [ ]:
import altair as alt
import pandas as pd
import polars as pl

from math import log10

#file = "/master/abagwell/variant-analysis/results/rhesus/stats/kernel_model/SHAPEIT5_merged/SNPRC_WGS_WES.SNP.kernel_model.skato.SkatO.assoc"
file = "/master/abagwell/variant-analysis/results/rhesus/stats/burden_test/SHAPEIT5_merged/SNPRC_WGS_WES.SNP.burden_test.CMC.CMC.assoc"
#file = "/master/abagwell/variant-analysis/results/rhesus/stats/single_variant_test/SHAPEIT5_merged/SNPRC_WGS_WES.SNP.single_variant.famLRT.FamLRT.assoc"
#file = "/master/abagwell/variant-analysis/results/rhesus/stats/single_variant_test/SHAPEIT5_merged/without_covar/SNPRC_WGS_WES.SNP.single_variant.famLRT.FamLRT.assoc"



table = pl.read_csv(file, separator="\t", infer_schema_length=10000).with_columns(
    pl.col("Pvalue").log10().alias("pval_log10").abs()  #.mul(-1)
)
# .filter(
#     pl.col("Pvalue") < max
# )
#.to_arrow().to_pandas()

line = alt.Chart(pd.DataFrame({'y': [-log10(0.05/table.shape[0])]})).mark_rule().encode(y='y')

In [ ]:
# For single-variant
table_modified = table.with_columns(pl.concat_str([
    pl.col("CHROM"),
    pl.lit(":"),
    pl.col("POS"),
    ]).alias("locus")
).filter(
    pl.col('pval_log10') > 2
)


alt.data_transformers.disable_max_rows()
plot = alt.Chart(table_modified).mark_circle().encode(
    alt.X("locus:O", title="Locus", axis=alt.Axis(labels=False, tickSize=0), sort=['1','2','3','4','5']),
    alt.Y("pval_log10:Q", title="-log10(p)", scale=alt.Scale(domain=[2, round(table_modified['pval_log10'].max() + 0.5)])),
    color=alt.Color("CHROM:N", title="CHROM", sort=['1','2','3','4','5']),
    tooltip=[
        alt.Tooltip("locus:O", title="Locus"),
        alt.Tooltip("pval_log10:Q", title="-log10(p)"),
    ],
).properties(
    title="FamLRT Single-Variant Model in SNPRC Rhesus (WGS and WES)",
    width=500,
)

plot + line

#(plot + line).save("/master/abagwell/figures/rare_variants/WGS_WES.famLRT.hr.trimmed.html")

In [ ]:
# For range methods (kernel model and burden test)
table_modified = table.with_columns(
    pl.col("RANGE").str.split(":").list.get(0).alias("CHROM")
)

In [ ]:
# For range - kernel model
max = 1

alt.data_transformers.disable_max_rows()
chart = alt.Chart(table_modified).mark_circle().encode(
    alt.X("RANGE:O", title="Interval", axis=alt.Axis(labels=False, tickSize=0), sort=['1','2','3','4','5']),
    alt.Y("pval_log10:Q", title="-log10(p)"),  #scale=alt.Scale(domain=[0, max])
    color=alt.Color("CHROM:N", title="CHROM", sort=['1','2','3','4','5']),
    tooltip=[
        alt.Tooltip("Range:O", title="Ensembl Gene ID"),
        alt.Tooltip("RANGE:O", title="Interval"),
        alt.Tooltip("pval_log10:Q", title="-log10(p)"),
    ],
).properties(
    title="SKAT-O Kernel Model in SNPRC Rhesus (WGS and WES)",
    width=500,
)

chart + line

#(chart + line).save("/master/abagwell/figures/rare_variants/WGS_WES.SKATO.hr.html")

In [ ]:
# For range - burden test
max = 1

alt.data_transformers.disable_max_rows()
chart = alt.Chart(table_modified).mark_circle().encode(
    alt.X("RANGE:O", title="Interval", axis=alt.Axis(labels=False, tickSize=0), sort=['1','2','3','4','5']),
    alt.Y("pval_log10:Q", title="-log10(p)"),  #scale=alt.Scale(domain=[0, max])
    color=alt.Color("CHROM:N", title="CHROM", sort=['1','2','3','4','5']),
    tooltip=[
        alt.Tooltip("Range:O", title="Ensembl Gene ID"),
        alt.Tooltip("RANGE:O", title="Interval"),
        alt.Tooltip("pval_log10:Q", title="-log10(p)"),
    ],
).properties(
    title="CMC Burden Test in SNPRC Rhesus (WGS and WES)",
    width=500,
)

chart + line
#(chart + line).save("/master/abagwell/figures/rare_variants/WGS_WES.CMC.hr.html")